In [1]:
# Import statements
import yfinance as yf
import pandas as pd
import os
import numpy as np
from datetime import date
from pathlib import Path
import pandas_ta as ta
from price_data_pull_yfinance import gethistoricalOHLC, saveHistStockData, loadHistDataFromDisk

from global_vars import dataPathToOHLC, pathToMasterDF, float16Cols, float32Cols

  TICKER FIRST_DATE_OHLC LAST_DATE_OHLC  \
0   MSFT      1999-12-31     2018-12-28   
1   AAPL      1999-12-31     2015-12-30   

                                            FILEPATH  
0  C:\Users\Philipp\Google Drive\Code_for_bothCom...  
1  C:\Users\Philipp\Google Drive\Code_for_bothCom...  
before update
updated
  TICKER FIRST_DATE_OHLC LAST_DATE_OHLC  \
0   MSFT      1999-12-31     2018-12-28   
1   AAPL      1999-12-31     2021-03-19   

                                            FILEPATH  
0  C:\Users\Philipp\Google Drive\Code_for_bothCom...  
1  C:\Users\Philipp\Google Drive\Code_for_bothCom...  


In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
masterDf = pd.read_csv(pathToMasterDF);
masterDf.head()

,TICKER,FIRST_DATE_OHLC,LAST_DATE_OHLC,FILEPATH
0,MSFT,1999-12-31,2018-12-28,C:\Users\Philipp\Google Drive\Code_for_bothCom...
1,AAPL,1999-12-31,2021-03-19,C:\Users\Philipp\Google Drive\Code_for_bothCom...


## Load the ticker symbols from the S&P 500 and download them via yahoo finance

The ticker symbols for the SnP500 are saved in the SnP500_constituents.csv. 

### Survivorship bias: 
Unfortunately, by using the latest version of the S&P500 we introduce a strong survivorship bias into our data sources. However, we are not able to optain more historical data. 
But, we are interested in the short term performance of stock. So that should be ok. 

In [27]:
from tqdm import tqdm
from time import sleep

In [7]:
SPY_ticker = pd.read_csv('../01_data/SnP500_constituents.csv');
SPY_ticker = SPY_ticker.iloc[:5]

In [31]:
tickerlist = SPY_ticker.Symbol.to_list();
tickerlist

['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD']

In [37]:
def batch_download(tickerlist):
    '''Download and save the tickerdata from yahoo finance and save them on disk.'''
    for ticker in tqdm(tickerlist):
        tickerDF = gethistoricalOHLC(ticker)
        saveHistStockData(ticker, tickerDF)
        sleep(0.3)

In [39]:
batch_download(tickerlist)

  0%|          | 0/5 [00:00<?, ?it/s]

before update
updated


 20%|██        | 1/5 [00:00<00:03,  1.03it/s]

before update
updated


 40%|████      | 2/5 [00:02<00:03,  1.02s/it]

...

 60%|██████    | 3/5 [00:03<00:02,  1.01s/it]

before update
updated


 80%|████████  | 4/5 [00:03<00:00,  1.18it/s]

before update
updated


100%|██████████| 5/5 [00:04<00:00,  1.13it/s]


# helper functions

In [22]:
def gethistoricalOHLC(ticker, start_date='1990-01-01', end_date=None):
    """finds a ticker and its representative OHLC data from yahoo finance.
    Returns a pandas dataframe"""

    if ticker is None:
        print("Ticker is empty!!!")

    if end_date == "" or end_date is None:
        end_date = str(date.today())

    # TODO #1 #create a checkup if that data is already in the database

    # instantiate the yf obj
    tickerObj = yf.Ticker(ticker)

    # get historical data
    df = tickerObj.history(start=start_date, end=end_date, interval="1d")

    df.reset_index(inplace=True)
    # cast column types
    float16TypeCast = [col for col in df.columns if col in float16Cols]
    float32TypeCast = [col for col in df.columns if col in float32Cols]
    df["Date"] = df["Date"].dt.date
    df[float16TypeCast] = df[float16TypeCast].astype("float16")
    df[float32TypeCast] = df[float32TypeCast].astype("float32")

    return df